In [ ]:
import pandas as pd

from sklearn.metrics.classification import accuracy_score
from sklearn.gaussian_process import GaussianProcessClassifier as GPC
from sklearn.gaussian_process.kernels import RBF, DotProduct as DP

from sklearn.model_selection import KFold

import warnings; warnings.simplefilter('ignore')

#  Import Data

In [ ]:
train = pd.read_csv('/Users/chuyiyu/Desktop/UCI HAR Dataset/train.csv')
test = pd.read_csv('/Users/chuyiyu/Desktop/UCI HAR Dataset/test.csv')

### separating data inputs and output lables

In [ ]:
trainData  = train.drop('Activity', axis=1).values
trainLabel = train.Activity.values
testData  = test.drop('Activity', axis=1).values
testLabel = test.Activity.values

# Gaussian Process Classification

### Instanciate a Gaussian Process model using RBF kernel

### Instanciate a Gaussian Process model using Dot Product kernel

In [ ]:
kernel_dp = 1.0 * DP(sigma_0=1.0)**2
gpc_dp = GPC(kernel=kernel_dp).fit(trainData, trainLabel)

### Log Marginal Likelihood on Training Set

In [ ]:
print("Log Marginal Likelihood : %.3f (RBF) %.3f (Dot Product)"
      % (gpc_rbf.log_marginal_likelihood(gpc_rbf.kernel_.theta),
      gpc_dp.log_marginal_likelihood(gpc_dp.kernel_.theta)))

### Prediction Accuracy on Test Set

In [ ]:
print("Accuracy (RBF): %.3f (train data) %.3f (test data)"
      % (accuracy_score(trainLabel, gpc_rbf.predict(trainData)),
         accuracy_score(testLabel, gpc_rbf.predict(testData))))

print("Accuracy (Dot Product): %.3f (train data) %.3f (test data)"
      % (accuracy_score(trainLabel, gpc_dp.predict(trainData)),
         accuracy_score(testLabel, gpc_dp.predict(testData))))

# Compare the Predictions using RBF and Dot Product Kernels via 10-fold Cross-validation with Log Marginal Likelihood as Error Measure

### Merging the training data and test data

In [ ]:
fulldata = train.append(test)
fulldata_X = fulldata.drop('Activity', axis=1).values
fulldata_y = fulldata.Activity.values

### Cross-validation

In [ ]:
n_splits=10
val_LML_rbf, val_LML_dp=0, 0
cv = KFold(n_splits=n_splits)
for train, test in cv.split(fulldata_X , fulldata_y):
    cv_rbf=GPC(kernel=kernel_rbf).fit(fulldata_X[train], fulldata_y[train])
    cv_dp=GPC(kernel=kernel_dp).fit(fulldata_X[train], fulldata_y[train])
    val_LML_rbf += cv_rbf.log_marginal_likelihood(cv_rbf.kernel_.theta)
    val_LML_dp += cv_dp.log_marginal_likelihood(cv_dp.kernel_.theta)
val_LML_rbf /= n_splits
val_LML_dp /= n_splits
print("log marginal likelihood: %.3f (RBF) %.3f (Dot Product)" % (val_LML_rbf,val_LML_dp))